# DATA COLLECTION FROM INSTAGRAM
### FINAL PROJECT FOR DATA COLLECTION TECHNIQUES - LUCÍA COLÍN COSANO

Firstly, we import the necessary libraries

In [7]:
import json
from lxml import html
import re
import csv
import numpy as np

from selenium import webdriver
import time
import os
import time
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (NoSuchElementException,TimeoutException,WebDriverException,)

 Chrome driver is set up. 

In [8]:
WAIT_TIME = 10

# Chrome driver should be in
executable_path=os.path.join('C:/Program Files/Google/Chrome/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-notifications')

preferences = {
    "profile.default_content_setting_values.notifications" : 1,
    "profile.default_content_setting_values.location": 1,
    "profile.managed_default_content_settings.images": 1,
    }
options.add_experimental_option("prefs", preferences)

chrome_options = Options()
chrome_options.add_argument("--disable-infobars")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")

driver = webdriver.Chrome(executable_path = executable_path,chrome_options = chrome_options,)

C:\Users\lulac\AppData\Local\Temp\ipykernel_1580\1192129472.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = executable_path,chrome_options = chrome_options,)
C:\Users\lulac\AppData\Local\Temp\ipykernel_1580\1192129472.py:22: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = executable_path,chrome_options = chrome_options,)


En ocasiones se muestra el error de la imagen inferior, que se soluciona añadiendo la siguiente instrucción:
    
driver = webdriver.Chrome(ChromeDriverManager().install())

<img src="error1.png">

Se maximiza la pantalla para trabajar mejor. El número 20 define un tiempo de espera.

In [9]:
driver.maximize_window()
driver.implicitly_wait(20) 

Una vez se ha abierto el navegador se define de que url se desean obtener los datos. En este caso el objetivo es poder sacar datos a partir de un hashtag de instagram.

In [10]:
driver.wait = WebDriverWait(driver, WAIT_TIME)
url = 'https://www.instagram.com/?hl=es'
driver.get(url)

Una vez se entra en instagram es necesario aceptar las cookies.

In [11]:
cookies_button = driver.find_element(By.CSS_SELECTOR,"button[class='_a9-- _a9_1']")
cookies_button.click()
time.sleep(1)

Se definen tanto usuario como contraseña. Se definen un comentarios unos input que se podria utilizar para pedir al usuario que introduzca sus credenciales. En este caso se dan directament escritas para comprobar el funcionamiento.

Se busca el boton en el que se ha de pulsar para introducir las credenciales y se inicia sesion.

In [12]:
USERNAME = 'lccunav'
PASSWORD = 'Tecnicadereccogidadedatos7'
#USERNAME = input('Introduzca su usuario:')
#PASSWORD= input('Introduzca su contraseña:')

usr = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.NAME, "username")))
usr.send_keys(USERNAME)

password = WebDriverWait(driver, WAIT_TIME).until(EC.element_to_be_clickable((By.NAME, "password")))
password.send_keys(PASSWORD)
time.sleep(4)

El siguiente error encontrado se debe a la definicion de las funciones que proporciona selenium para la seleccion de elementos. 
Estas antes seguian el siguiente formato: driver.find_elements_by_css_selector, driver.find_elements_by_xpath.
Esta libreria ha evolucionado, generalizando esta función a **driver.find_elements()** y el primer argumento de la funcion es el que determina que tipo de busqueda se quiere realizar.

Es necesario impotar **from selenium.webdriver.common.by import By**.
Dos ejemplos de implementación utilizados son:

- driver.find_element(By.CSS_SELECTOR,"button[class='_a9-- _a9_1']")
- driver.find_element(By.XPATH, '//button[text()="._9AH"]')

<img src="error2.png">

In [13]:
driver.find_elements(By.CSS_SELECTOR,('button'))[1].click()
time.sleep(2)

Una vez se inicia sesion, aparece un recuadro sobre si se desea guardar la información, y se pulsa el boton de "Ahora no". Y con la activación de las notificaciones igual. 

In [14]:
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[class='_acan _acap _acas _aj1-']")))
guardar_info_button=driver.find_element(By.CSS_SELECTOR,"button[class='_acan _acap _acas _aj1-']")
guardar_info_button.click()
time.sleep(2)

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"button[class='_a9-- _a9_1']")))
notificaciones_button=driver.find_element(By.CSS_SELECTOR,"button[class='_a9-- _a9_1']")
notificaciones_button.click()
time.sleep(5)

Se define el hashtag que se desean scrapear y se busca la barra de busqueda de instagram.

In [15]:
HASHTAG='#mundialqatar2022'

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"a[href='#']")))
busqueda_button=driver.find_element(By.CSS_SELECTOR,"a[href='#']")
busqueda_button.click()

WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[aria-label='Buscar entrada']")))
intro_hashtag=driver.find_element(By.CSS_SELECTOR,"input[aria-label='Buscar entrada']")
intro_hashtag.clear()
intro_hashtag.send_keys(HASHTAG)
time.sleep(5)
intro_hashtag.send_keys(Keys.ENTER)
time.sleep(5)
intro_hashtag.send_keys(Keys.ENTER)
time.sleep(5)

Una vez nos encontramos con las publicaciones correspondientes al hashtag seleccionado, se quiere obtener el link de la publicacion, el usuario que publica, la localizacion (que en númerosas ocasiones no está disponible), el tiempo desde que se realizó la publicación, el número de likes y finalmente el pie de foto.

In [16]:
enlace_post = driver.find_elements(By.TAG_NAME,'a')
post = [i.get_attribute('href') for i in enlace_post]
print(len(post))

url_post=[]
for i in post:
    if '/p/' in i:
        url_post.append(i)  

45


Se recorren las url de las publicaciones que se han obtenido y en el siguiente bucle se accede a cada post para obtener la información correspondiente. Además se define el número de publicaciones de las que se desea obtener la info. Se recomienda que este no sea muy elevado ya que conllevaria altos tiempos de espera.

In [17]:
users=[] #usuarios de los post
times=[] #fecha en la que se publicó el post
locations=[] #localizaciones de los post
likes=[] #numero de likes
pie=[] #pie de foto

num_publi_deseadas=4

#for i in url_post:
for i in url_post[0:num_publi_deseadas]:

    driver.get(i)
    
    #Para obtener los usuarios en una lista
    try:
        users.append(driver.find_element(By.CSS_SELECTOR, "a[class='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _acan _acao _acat _acaw _aj1- _a6hd']").text)
    except NoSuchElementException:
        users.append("Publicación no disponible")
        
    #Para obtener las localizaciones en una lista    
    try:
        locations.append(driver.find_element(By.CSS_SELECTOR, "a[class='x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz _aaqk _a6hd']").text)
    except:
        locations.append(" ")
        
    #Para obtener el tiempo transcurrido desde que se publicó en una lista    
    try:
        times.append(driver.find_element(By.CSS_SELECTOR,"time[class='_aaqe']").text)     
    except:
        times.append("Publicación no disponible")  
    
    #Para obtener el número de likes que hay en cada publicación en una lista
    try:
        likes.append(driver.find_element(By.CSS_SELECTOR,"div[class='_aacl _aaco _aacw _aacx _aada _aade']").text)
    except:
        likes.append(' ')
        
    #Para obtener el pie de foto en una lista
    try:
        pie.append(driver.find_element(By.CSS_SELECTOR,"div[class='_a9zr']").text)
    except:
        pie.append(' ')
    time.sleep(3)

Finalmente se crea el dataframe y se imprime.

In [18]:
df = pd.DataFrame(list(zip(url_post,users,locations, times, likes,pie)), columns=['Link al Post','Nombre del Usuario', 'Localización', 'Tiempo desde la publiación', 'Número de likes/reproducciones','Pie de foto'])
df

,Link al Post,Nombre del Usuario,Localización,Tiempo desde la publiación,Número de likes/reproducciones,Pie de foto
0,https://www.instagram.com/p/CmUoG38ADob/,leomessiandfamily,,HACE 21 HORAS,44.895 Me gusta,leomessiandfamily\nThis is Love 💙🥺\n\n#messi #...
1,https://www.instagram.com/p/CmWczJpOTmt/,daleseleccion,Lusail Stadium,HACE 4 HORAS,2091 Me gusta,daleseleccion\nNuestro Ángel de la guarda ❤️🇦🇷...
2,https://www.instagram.com/p/CmUe9GJukZl/,daleseleccion,Lusail Stadium,HACE 23 HORAS,14.996 Me gusta,daleseleccion\nEl sueño se hizo realidad\n\n#m...
3,https://www.instagram.com/p/CmWT_WrucV-/,depoweb,,HACE 6 HORAS,1521 Me gusta,"depoweb\n🤌 ANDA PA' ALLÁ, KYLIAN\n\n👋Luego de ..."


## **ERRORES GENERALIZADOS** 

En muchas ocasiones no era posible pulsar botones como los de aceptar notificaciones. Esto se debe a los tiempos de espera. Ha sido necesario jugar con estos valores hasta conseguir que nunca fallase.

Otro error que ha salido continuamente es InvalidSelectorException, como se muestra a continuación. Esto se debe a que en ocasiones el xpath especificado no es válido. Debido a los continuos fallos se decidió utilizar exclusivamente el css_selector.
<img src="error4.png">

Otro aspecto a resaltar es la importancia de la eleccion de **find_element** y **find_elements**. La diferencia entre estos es la siguiente. Es importante tenerlo en cuenta ya que si no aparece el error que se muestra en la imagen inferior.

- findElement: comando utilizado para identificar de forma única un elemento web dentro de la página web.
- findElements: comando utilizado para identificar una lista de elementos web dentro de la página web.

<img src="error6.png">